In [43]:
from flask import Flask
from flask import render_template,request
from datetime import datetime
import mysql.connector
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
def item_call(item_id,db):
    results=db.execute(f'call item_call({item_id})',multi=True)
    data=[]
    try:
        for i in results:
            data=i.fetchall()
    except:
        pass
    data=pd.DataFrame(data,columns=['order_date','order_hour','quantity'])
    data['order_day']=list(map((lambda x:x.weekday()),data['order_date'].to_list()))
    return data

In [ ]:
def predict(data,day,time):
    scaler=StandardScaler()
    data[['order_day','order_hour']]=scaler.fit_transform(data[['order_day','order_hour']])
    x_train,x_test,y_train,y_test=train_test_split(data[['order_day','order_hour']],data['quantity'],test_size=0.0,shuffle=True)
    linear=linear_model.LinearRegression()
    reg=linear.fit(x_train,y_train)
    result=reg.predict([day,time])
    result=result.round()
    return result

In [44]:
mydb=mysql.connector.connect(host="127.0.0.1",user="root", passwd="Govinda457", database='leftovers1')
my=mydb.cursor()

In [45]:
my.callproc('total_orders')

()

In [ ]:
app =Flask(__name__, template_folder='template') 

@app.route("/",methods = ['POST','GET'])  
def result():
  if request.method=='POST':
    date=request.form['mydate']
    time=request.form['mytime']
    item_id=request.form['item_id']
    day= datetime.strptime(date,'%Y-%m-%d').weekday()
    data=item_call(item_id,my)
    result=predict(data)
    return render_template('input.html', msg1 =result)  
  return render_template('input.html', message = "")

if __name__ =='__main__':  
    app.run(debug = True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2023 14:15:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 14:15:31] "GET /static/style2.css HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2023 14:15:32] "GET /static/minimal-geometric-stripe-shape-background_1409-1014.webp HTTP/1.1" 304 -
127.0.0.1 - - [30/Jan/2023 14:15:32] "GET /favicon.ico HTTP/1.1" 404 -
